In [1]:
import os
import pandas as pd
from src.processer_file import read_path 
from src.processer_file import get_path 
from src.processer_file import export_file

# export_file.export_to_csv(15, 'task')
# df_dict = reader.read(16, 'task')

# for k,v in df_dict.items():
#     print(k)
#     for k2, v2 in v.items():
#         print(f'{k2, v2.shape}')

export_file.export_to_csv(99, 'igv')

('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V001_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V002_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V003_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V004_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V005_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V006_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V008_IGVData_KPI.csv')
('ck20240408', 'c:\\Users\\westwell\\Documents\\

In [5]:
import os
import pandas as pd

root = 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99'
paths= []

for item in os.listdir(root):
    item_path = os.path.join(root, item)
    if os.path.isfile(item_path):
        paths.append(item_path)

paths

['C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_CK.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_DL.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_ICA.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TJ.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TPY.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TS.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_WH.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_YH.csv']

In [15]:
for i in range(1,len(paths)):
    df = pd.read_csv(paths[i], index_col= 0)
    df['loc_len'] = df['target_location'].str.len()
    col = ['loc_len', 'target_location']

    print()
    print(paths[i])
    print(df['target_location'].isna().value_counts())
    print(df[col].value_counts())

C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_DL.csv
target_location
False    77702
Name: count, dtype: int64
loc_len
17    47705
20    29997
Name: count, dtype: int64
C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_ICA.csv
target_location
False    16417
Name: count, dtype: int64
loc_len
26    15817
12      448
25      152
Name: count, dtype: int64
C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv
target_location
False    487113
Name: count, dtype: int64
loc_len
18    234946
16    181663
1      70504
Name: count, dtype: int64
C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TPY.csv
target_location
False    57463
Name: count, dtype: int64
loc_len
17    31421
20    25466
13      576
Name: count, dtype: int64
C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TS.csv
target_location
Fa